### Scraper for <a href="https://www.apconsulting.jp/%E3%83%96%E3%83%AD%E3%82%B0/"> APConsulting </a> website

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
import requests

In [22]:
from os import link


def get_chrome_Driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    return driver

def get_soup(URL):
    r = requests.get(URL)
    if r:
        driver = get_chrome_Driver()
        driver.get(URL)
        htmlContent = driver.page_source
        soup =  BeautifulSoup(htmlContent, 'html.parser')
        driver.close()
        return soup
    else:
        print("Error Fetching Soup, Scraping Ended")
        return False

def get_link_from_soup(soup, prefix=""):
    link_list = set()
    try:
        wanted_div = soup.findAll("div", {"class":"blogselection"})[0]
    except IndexError:
        print("HTML element not found, returning empty set")
        return set()
    for div in wanted_div.findAll("div", {"class":"n clearover j-blogarticle"}):
        wanted_article = div.find('a', href=True)["href"]
        link_list.add(prefix + wanted_article)
    return link_list

def save_link_as_txt(link_list):
    file_path = "links/apconsulting.txt"
    # if file exist, get its content, merge to existing set and then overwrite
    if os.path.isfile(file_path):
        with open(file_path, 'r+', encoding="utf-8") as file:
            # get existing links in file
            old_links = set(file.readlines())
            # remove duplicates 
            link_list = link_list - old_links
            file.seek(0)
            for i in link_list:
                file.write(i+"\n")
            file.truncate()
    else: # if not, make new file
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, "w", encoding="utf-8") as file:
            for i in link_list:
                file.write(i+"\n")
    print(f"Links Saved in {file_path}")
    return True

In [27]:
all_links = set()
soup = get_soup(f"https://www.apconsulting.jp/%E3%83%96%E3%83%AD%E3%82%B0/")
current_links = get_link_from_soup(soup, prefix = "https://www.apconsulting.jp")
if len(current_links) > 0:
        all_links.update(current_links)
else: # if tags we want are not found
        print("No such HTML Tags Found")

save_link_as_txt(all_links)

True